# Neural Network Traning

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from CONSTANTS import ML

print('---------------------------------------------------------------------------')
print('TF: {}'.format(tf.__version__))
print('---------------------------------------------------------------------------')

########################################################################
# Change file path HERE
########################################################################
csv_filepath = './data/2020-07-31_04-20-48.csv'
csv_filepath2 = './data/2020-07-26_06-35-07-unfiltered.csv'


---------------------------------------------------------------------------
TF: 2.2.0
---------------------------------------------------------------------------


# Compiling dataset features

Total number of features = 19

### Power Spectrum Bins

1, 2, 3, 4, 10, 11, 22, 23, 43, 44, 45, 109, 110, 125, 126

### MFCC Bins

1, 10

### Spectral Flatness

Spectral flatness value and delta spectral flatness

In [3]:
def extractFeatures(df):
    datasets = np.empty([df.shape[0], ML['NUMBER_OF_FEATURES']])

    for index, row in df.iterrows():
        sample = np.array([
            row['powerSpectrum_1'],
            row['powerSpectrum_2'],
            row['powerSpectrum_3'],
            row['powerSpectrum_4'],
            row['powerSpectrum_10'],
            row['powerSpectrum_11'],
            row['powerSpectrum_22'],
            row['powerSpectrum_23'],
            row['powerSpectrum_43'],
            row['powerSpectrum_44'],
            row['powerSpectrum_45'],
            row['powerSpectrum_109'],
            row['powerSpectrum_110'],
            row['powerSpectrum_125'],
            row['powerSpectrum_126'],
            row['mfcc_1'],
            row['mfcc_10'],
            row['spectralFlatness'],
            row['delta_spectralFlatness']
        ])
        datasets[index] = sample
    
    return datasets

# Data preprocessing

- Create one hot for the three classes we (Chloe, Beep, and Music)
- Feature normalization

In [4]:
def standardize(x, axes=[0], epsilon=1e-8):
    mean, variance = tf.nn.moments(x, axes=axes)
    x_normed = (x - mean) / tf.sqrt(variance + epsilon) # epsilon to avoid dividing by zero
    return x_normed


def preprocess(x, y):
    x = tf.cast(x, tf.float32)
    y = tf.cast(y, tf.int64)
    return x, y


def preprocess_standardized(x, y):
    x = tf.cast(x, tf.float32)
    x_normed = standardize(x)
    y = tf.cast(y, tf.int64)
    return x_normed, y


def create_dataset(xs, ys, n_classes=ML['N_CLASSES']):
    ys = tf.one_hot(ys, depth=n_classes)
    return tf.data.Dataset.from_tensor_slices((xs, ys)).map(preprocess).shuffle(len(ys)).batch(128)


def create_dataset_standardized(xs, ys, n_classes=ML['N_CLASSES']):
    ys = tf.one_hot(ys, depth=n_classes)
    return tf.data.Dataset.from_tensor_slices((xs, ys)).map(preprocess_standardized).shuffle(len(ys)).batch(128)

# Training

In [5]:
train_df = pd.read_csv(csv_filepath, sep=',', header=0)
validation_df = pd.read_csv(csv_filepath2, sep=',', header=0)

train_classes = train_df['class'].to_numpy()
validation_classes = validation_df['class'].to_numpy()


train_extract = extractFeatures(train_df)
validation_extract = extractFeatures(validation_df)

train_dataset = create_dataset(train_extract, train_classes)
validation_dataset = create_dataset(validation_extract, validation_classes)

model = keras.Sequential([
    keras.layers.Dense(units=22, activation='relu'),
    keras.layers.Dense(units=16, activation='relu'),
    keras.layers.Dense(units=ML['N_CLASSES'], activation='softmax')
])

model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_dataset.repeat(), 
    epochs=10, 
    steps_per_epoch=500,
    validation_data=validation_dataset.repeat(), 
    validation_steps=2
)

Epoch 1/10
500/500 [==============================] - 2s 3ms/step - loss: 1.0191 - accuracy: 0.5168 - val_loss: 0.6839 - val_accuracy: 0.8750
Epoch 2/10
500/500 [==============================] - 1s 3ms/step - loss: 0.5878 - accuracy: 0.9675 - val_loss: 0.6678 - val_accuracy: 0.8828
Epoch 3/10
500/500 [==============================] - 1s 3ms/step - loss: 0.5788 - accuracy: 0.9772 - val_loss: 0.6869 - val_accuracy: 0.8594
Epoch 4/10
500/500 [==============================] - 1s 3ms/step - loss: 0.5731 - accuracy: 0.9810 - val_loss: 0.6544 - val_accuracy: 0.8984
Epoch 5/10
500/500 [==============================] - 1s 3ms/step - loss: 0.5686 - accuracy: 0.9852 - val_loss: 0.6398 - val_accuracy: 0.9141
Epoch 6/10
500/500 [==============================] - 1s 3ms/step - loss: 0.5644 - accuracy: 0.9890 - val_loss: 0.6443 - val_accuracy: 0.9102
Epoch 7/10
500/500 [==============================] - 2s 4ms/step - loss: 0.5618 - accuracy: 0.9909 - val_loss: 0.6220 - val_accuracy: 0.9297
Epoch 